# ばばぬきパターン出力

In [11]:
import os
import random
import pandas as pd

COUNT_OF_SAMPLES = 100
CARD_SUITS = ["♡","♧","♤","♢"]
CARD_RANKS = ["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
CARD_JOKER = ["☆"]
ALL_PAIR_P = 10  # 全員がペアを持つケース
NO_PAIR_P = 10  # 全員がペアを持たないケース
PLAYER_COUNT = 6

In [12]:
def create_all_cards():
    """53枚のカードを生成"""
    all_cards = []
    for suit in CARD_SUITS:
        for rank in CARD_RANKS:
            all_cards.append(suit + rank)
    all_cards.append('☆')
    return all_cards
all_cards = create_all_cards()
all_cards

['♡A',
 '♡2',
 '♡3',
 '♡4',
 '♡5',
 '♡6',
 '♡7',
 '♡8',
 '♡9',
 '♡10',
 '♡J',
 '♡Q',
 '♡K',
 '♧A',
 '♧2',
 '♧3',
 '♧4',
 '♧5',
 '♧6',
 '♧7',
 '♧8',
 '♧9',
 '♧10',
 '♧J',
 '♧Q',
 '♧K',
 '♤A',
 '♤2',
 '♤3',
 '♤4',
 '♤5',
 '♤6',
 '♤7',
 '♤8',
 '♤9',
 '♤10',
 '♤J',
 '♤Q',
 '♤K',
 '♢A',
 '♢2',
 '♢3',
 '♢4',
 '♢5',
 '♢6',
 '♢7',
 '♢8',
 '♢9',
 '♢10',
 '♢J',
 '♢Q',
 '♢K',
 '☆']

In [19]:
def _init_cards():
    available_cards = all_cards.copy()
    players_hands = [[] for _ in range(6)]

    # 8枚を受け取るプレイヤーをランダムに決定
    player_with_eight = random.randint(0, PLAYER_COUNT-1)
    
    # ジョーカーをランダムなプレイヤーに配布
    joker_player = random.randint(0, PLAYER_COUNT-1)
    players_hands[joker_player] += CARD_JOKER
    available_cards.remove(CARD_JOKER[0])
    return available_cards, players_hands, player_with_eight
_init_cards()

(['♡A',
  '♡2',
  '♡3',
  '♡4',
  '♡5',
  '♡6',
  '♡7',
  '♡8',
  '♡9',
  '♡10',
  '♡J',
  '♡Q',
  '♡K',
  '♧A',
  '♧2',
  '♧3',
  '♧4',
  '♧5',
  '♧6',
  '♧7',
  '♧8',
  '♧9',
  '♧10',
  '♧J',
  '♧Q',
  '♧K',
  '♤A',
  '♤2',
  '♤3',
  '♤4',
  '♤5',
  '♤6',
  '♤7',
  '♤8',
  '♤9',
  '♤10',
  '♤J',
  '♤Q',
  '♤K',
  '♢A',
  '♢2',
  '♢3',
  '♢4',
  '♢5',
  '♢6',
  '♢7',
  '♢8',
  '♢9',
  '♢10',
  '♢J',
  '♢Q',
  '♢K'],
 [[], [], ['☆'], [], [], []],
 5)

In [17]:
def _rank_counts(cards):
    ranks_and_joker = CARD_RANKS + CARD_JOKER
    result = {raj:0 for raj in ranks_and_joker}
    for c in cards:
        if len(c)==1:
            result[c] += 1
        else:
            result[c[1:]] += 1
    return result

_rank_counts(all_cards)

{'A': 4,
 '2': 4,
 '3': 4,
 '4': 4,
 '5': 4,
 '6': 4,
 '7': 4,
 '8': 4,
 '9': 4,
 '10': 4,
 'J': 4,
 'Q': 4,
 'K': 4,
 '☆': 1}

In [37]:
def _distribute_remaining_cards(available_cards, players_hands, player_with_eight):
    while available_cards:
        for i in range(PLAYER_COUNT):
            target_size = 8 if i == player_with_eight else 9
            if len(players_hands[i]) < target_size:
                card = random.choice(available_cards)
                players_hands[i].append(card)
                available_cards.remove(card)

    # シャッフルし直して返す
    for i in range(PLAYER_COUNT):
        random.shuffle(players_hands[i])
    return available_cards, players_hands, player_with_eight


In [69]:
def create_all_random_case():
    available_cards, players_hands, player_with_eight = _init_cards()

    # 残りのカードでランダム取得して充填する
    available_cards, players_hands, player_with_eight = _distribute_remaining_cards(available_cards, players_hands, player_with_eight)
    
    return players_hands
create_all_random_case()

[['♢9', '♢5', '♤K', '♧Q', '♡8', '♤3', '♤5', '♧K'],
 ['♡3', '♤Q', '♧4', '♡6', '♧A', '♤J', '♧J', '♡5', '♢10'],
 ['♢6', '♡2', '♧9', '♤7', '♧6', '♧7', '♡4', '♢2', '♡K'],
 ['♡Q', '♡7', '♢3', '♤A', '♢J', '♤10', '♧8', '♢A', '♤8'],
 ['♤6', '♢K', '♡A', '♡9', '♧2', '♢7', '♧5', '♢Q', '☆'],
 ['♡J', '♤2', '♧10', '♡10', '♤9', '♤4', '♢8', '♢4', '♧3']]

In [68]:
def create_all_pairs_case():
    """全員がペアを持つテストケースを生成"""
    available_cards, players_hands, player_with_eight = _init_cards()

    # まず各プレイヤーにペアを配布
    for i in range(PLAYER_COUNT):

        # 使用可能なランクを取得
        ranks = list(set(rank for rank,count in _rank_counts(available_cards).items() if count>1))
        
        # ランダムにランクを選択
        selected_rank = random.choice(ranks)
        pair_cards = []
        for card in available_cards:
            if card[1:] == selected_rank:
                pair_cards.append(card)
                if len(pair_cards)==2:
                    break

        selected_pair = random.sample(pair_cards, 2)
        
        # 手札に追加
        players_hands[i].extend(selected_pair)
        for card in selected_pair:
            available_cards.remove(card)
       
    # 残りのカードでランダム取得して充填する
    available_cards, players_hands, player_with_eight = _distribute_remaining_cards(available_cards, players_hands, player_with_eight)
    
    return players_hands
    
players_hands = create_all_pairs_case()
players_hands

[['♡J', '♧J', '♢8', '♢9', '☆', '♤8', '♧6', '♡6', '♤K'],
 ['♧4', '♤Q', '♡A', '♤6', '♧3', '♧5', '♤3', '♡4'],
 ['♡8', '♧8', '♤5', '♤2', '♢6', '♤7', '♧K', '♢3', '♡9'],
 ['♢10', '♢A', '♧2', '♡2', '♡3', '♤A', '♢2', '♧9', '♡7'],
 ['♢4', '♢J', '♧A', '♤J', '♧7', '♡10', '♡5', '♢5', '♤10'],
 ['♢K', '♡K', '♤4', '♧Q', '♢Q', '♡Q', '♢7', '♤9', '♧10']]

In [54]:
src_p, dst_p = random.sample(range(0,PLAYER_COUNT),2)
src_p, dst_p

(3, 2)

In [66]:
def create_no_pairs_case():
    """全員がペアを持たないテストケースを生成"""
    available_cards, players_hands, player_with_eight = _init_cards()
    
    # 各プレイヤーにカードを配布していく。プレイヤー順に昇順にカードを与えていくことで、重複（ペア）が起きないようにしている
    rank_id = 0
    while available_cards:
        for i in range(PLAYER_COUNT):
            target_card_count = 8 if i == player_with_eight else 9
            while len(players_hands[i]) < target_card_count:
                
                # 使用可能なランクを取得
                target_rank = CARD_RANKS[rank_id]
                target_rank_cards = [card for card in available_cards if card[1:]==target_rank]
                get_card = random.sample(target_rank_cards, 1)[0]
                
                # 手札に追加
                players_hands[i].append(get_card)
                available_cards.remove(get_card)

                # 次のrankへ
                rank_id += 1
                rank_id %= len(CARD_RANKS)

    # プレイヤー間でシャッフルする
    players_hands2 = players_hands.copy()
    random.shuffle(players_hands2)
    
    # 1000回シャッフルする
    for random_id in range(1000):
        src_p, dst_p = random.sample(range(PLAYER_COUNT),2)
        src_ranks = set([hand[1:] for hand in players_hands2[src_p] if len(hand)>1])
        dst_ranks = set([hand[1:] for hand in players_hands2[dst_p] if len(hand)>1])
        src_ranks2 = src_ranks - dst_ranks  # 重複を取り除く
        dst_ranks2 = dst_ranks - src_ranks  # 重複を取り除く
        if len(src_ranks2)>0 and len(dst_ranks2)>0:
            # print("="*50)
            # print(f"players_hands2[{src_p}]=",players_hands2[src_p],f", players_hands2[{dst_p}]=",players_hands2[dst_p])
            # print("src_ranks2=",src_ranks2,", dst_ranks2=",dst_ranks2)
            src_hands = [hand for hand in players_hands2[src_p] if hand[1:] in src_ranks2] # 両者間でランクの重複を取り除いたカードを取得
            dst_hands = [hand for hand in players_hands2[dst_p] if hand[1:] in dst_ranks2] # 両者間でランクの重複を取り除いたカードを取得
            # print("src_hands=",src_hands,", dst_hands=",dst_hands)
            src_remove = random.sample(src_hands,1)[0]  # 移動する候補を選択
            dst_remove = random.sample(dst_hands,1)[0]  # 移動する候補を選択
            src_i = players_hands2[src_p].index(src_remove)
            dst_i = players_hands2[dst_p].index(dst_remove)
            players_hands2[dst_p].append(players_hands2[src_p].pop(src_i)) # 移動
            players_hands2[src_p].append(players_hands2[dst_p].pop(dst_i)) # 移動
        else:
            print(f'移動可能なデータがありません random_id={random_id}')

    return players_hands2

players_hands = create_no_pairs_case()
players_hands

[['♤9', '♢5', '♤8', '♧4', '♤7', '♡2', '♧Q', '♡A', '♧K'],
 ['♢3', '♢4', '♡6', '♡9', '♤K', '♢2', '♤J', '♧5', '♢Q'],
 ['♡10', '♧3', '♡7', '♡8', '♧2', '♡5', '♧9', '♢A', '♡J'],
 ['♧A', '♡3', '♧J', '♧10', '♤6', '♧7', '♤4', '♧8'],
 ['♢9', '♤5', '♡K', '♢10', '♤2', '♢7', '♢8', '♢6', '♤Q'],
 ['☆', '♤3', '♡Q', '♢K', '♤A', '♧6', '♡4', '♢J', '♤10']]

In [80]:
def get_random_inputs(not_list=[]):
    while True:
        not_list_index = len(not_list)

# ALL_PAIR_P = 10  # 全員がペアを持つケース
# NO_PAIR_P = 10  # 全員がペアを持たないケース
        
        if not_list_index <= ALL_PAIR_P:  # 全員がペアを持つケース
            test_case = create_all_pairs_case()
        elif not_list_index <= ALL_PAIR_P+NO_PAIR_P:  # 全員がペアを持たないケース
            test_case = create_no_pairs_case()
        else:
            test_case = create_all_random_case()

        inputs = [" ".join(line1) for line1 in test_case]
        if inputs in not_list:
            print("重複", choice)
            continue

        return inputs
    raise ValueError('無限ループしたようです')
    
#get_random_inputs()

In [83]:
def solve(input_list):
    #print(input_list)
    # 6人分の手札を入力
    hands = []
    for i in range(6):
        hand = input_list[i].split()
        hands.append(hand)
    
    # 結果を格納するリスト
    pair_results = []
    
    # 各プレイヤーのペアとジョーカーを判定
    joker_player = 0
    for i in range(6):
        # ジョーカー判定
        if '☆' in hands[i]:
            joker_player = i + 1
            hands[i].remove('☆')
        
        # ペア判定用のリスト
        ranks = []
        for card in hands[i]:
            ranks.append(card[1:])  # スートを除いたランク部分を追加
        
        # ペア判定
        has_pair = False
        for rank in ranks:
            if ranks.count(rank) >= 2:
                has_pair = True
                break
        
        # 結果を追加
        if has_pair:
            pair_results.append("Yes")
        else:
            pair_results.append("No")
    
    # 結果を出力
    output_list = [
        " ".join(pair_results),
        str(joker_player)
    ]
    return output_list

In [84]:
def output_all(out_xz_filename):
    all_inputs = []
    all_outputs = []

    for answer_id in range(COUNT_OF_SAMPLES):
        all_inputs.append(get_random_inputs(all_inputs))
        all_outputs.append(solve(all_inputs[answer_id]))        # 解答を求める

    df_all = pd.DataFrame({"in":all_inputs, "out":all_outputs})
    df_all.index += 1
    os.makedirs(os.path.dirname(out_xz_filename), exist_ok=True)
    df_all.to_csv(out_xz_filename)

output_all("data/answer.csv.xz")
print('done')

移動可能なデータがありません random_id=0
移動可能なデータがありません random_id=489
移動可能なデータがありません random_id=490
移動可能なデータがありません random_id=522
移動可能なデータがありません random_id=0
移動可能なデータがありません random_id=0
移動可能なデータがありません random_id=5
done


In [86]:
df_answer = pd.read_csv("data/answer.csv.xz", index_col=0)
df_answer

,in,out
1,"['♡J ♤3 ♡2 ♤10 ♡10 ♧2 ♢5 ♡7 ☆', '♧9 ♧6 ♡9 ♤2 ♤...","['Yes Yes Yes Yes Yes Yes', '1']"
2,"['♧10 ♧9 ♡8 ♡7 ♤K ♤5 ♤Q ♧8', '♤10 ♡2 ♤7 ♧J ♧4 ...","['Yes Yes Yes Yes Yes Yes', '4']"
3,"['♡K ♢5 ♤Q ♡Q ♧7 ♧Q ♢A ♢K', '♧4 ♡4 ♢4 ♡10 ♢10 ...","['Yes Yes Yes Yes Yes Yes', '3']"
4,"['♤7 ♧5 ♡7 ♧4 ♤Q ☆ ♡4 ♤2 ♢K', '♡10 ♢9 ♢8 ♧10 ♧...","['Yes Yes Yes Yes Yes Yes', '1']"
5,"['♡4 ♧2 ♧7 ♧8 ♡K ♤6 ♡8 ♢10 ♤3', '♤9 ♢A ♡7 ♢2 ♤...","['Yes Yes Yes Yes Yes Yes', '2']"
...,...,...
96,"['♤5 ♤A ♤Q ♧10 ♧4 ♢8 ♡6 ♧Q ☆', '♡7 ♡3 ♤8 ♤2 ♡4...","['Yes Yes Yes Yes Yes Yes', '1']"
97,"['♧4 ♧7 ♢7 ♡10 ♡4 ♡9 ♤A ♤5 ♧A', '♧3 ♡2 ♢10 ♧8 ...","['Yes Yes Yes Yes Yes Yes', '6']"
98,"['♧3 ♤7 ♡9 ♤4 ♧10 ♧2 ♡6 ♢5 ♢8', '♢3 ♡J ♤K ♧4 ♢...","['No Yes Yes Yes Yes Yes', '3']"
99,"['♡2 ♢5 ♢2 ♢8 ♢10 ♡8 ♢A ♤8 ♤7', '♤4 ♢7 ♧5 ♡Q ♧...","['Yes No Yes Yes Yes No', '6']"


In [94]:
def output_answer (answer_id):
    for input1 in eval(df_answer.loc[answer_id]['in']):
        print(input1)
    
    for output1 in eval(df_answer.loc[answer_id]['out']):
        print(output1)

# すべてNoのケース
output_answer(12)

♤Q ♡4 ♧J ♢2 ♡3 ♤7 ♡5 ♡9 ♤K
☆ ♡2 ♡7 ♤10 ♤6 ♤8 ♤5 ♧A ♧4
♧10 ♧3 ♢8 ♡A ♢4 ♡Q ♧K ♧2 ♢7
♢10 ♢J ♢K ♤4 ♢Q ♡6 ♤9 ♢5
♡K ♢9 ♡J ♧7 ♤A ♡8 ♢3 ♧Q ♧6
♧8 ♢6 ♤J ♡10 ♤3 ♧5 ♢A ♧9 ♤2
No No No No No No
2


In [96]:
df_answer[50:60]

,in,out
51,"['♧9 ♡6 ♤K ♧7 ♤10 ♧5 ♡8 ♢7 ♢5', '♤9 ♡Q ♢10 ♤5 ...","['Yes Yes Yes Yes Yes Yes', '3']"
52,"['♡4 ♡7 ♤10 ♢A ♧Q ♢K ☆ ♢6 ♤8', '♢7 ♡6 ♧6 ♧7 ♤A...","['No Yes Yes Yes Yes No', '1']"
53,"['♧J ♡Q ♤3 ♢K ♢J ♤K ♧10 ♤6 ♧6', '♢5 ♡3 ♧Q ♡A ♢...","['Yes No No Yes Yes Yes', '4']"
54,"['♡5 ♤A ♤K ♤5 ♡6 ♢4 ♧A ♡2 ♢2', '♢A ♡Q ♡7 ♧5 ♢7...","['Yes Yes Yes Yes Yes Yes', '4']"
55,"['♡Q ♢Q ♤6 ♡3 ♧Q ♤3 ♧10 ♤A ♡10', '♢J ♤4 ♧7 ♧A ...","['Yes Yes Yes Yes Yes Yes', '5']"
56,"['♧4 ♧K ♤5 ♧2 ♤3 ♡10 ♧J ♢Q ♧7', '♡9 ♡4 ♡7 ♧6 ♤...","['No Yes Yes Yes Yes Yes', '5']"
57,"['♡2 ♡K ♡3 ♧10 ♧4 ♧6 ♤J ♧K ♤4', '♤8 ♢2 ♢7 ♢10 ...","['Yes Yes Yes Yes Yes Yes', '4']"
58,"['♡A ♤2 ♡10 ♡9 ♢5 ♤8 ♤5 ♧2', '♧5 ♧7 ♧A ♧Q ♧3 ♡...","['Yes Yes Yes Yes Yes Yes', '5']"
59,"['♧J ♤J ♧5 ♤3 ☆ ♧Q ♡6 ♢9 ♢3', '♧3 ♢A ♤9 ♧10 ♡Q...","['Yes Yes Yes Yes Yes Yes', '1']"
60,"['♡9 ♢9 ♧8 ♧A ♡8 ♧3 ♢2 ☆ ♡7', '♤9 ♡J ♧2 ♤A ♢8 ...","['Yes No Yes No Yes Yes', '1']"


In [97]:
# Yes・Noが混在しているケース
output_answer(60)

♡9 ♢9 ♧8 ♧A ♡8 ♧3 ♢2 ☆ ♡7
♤9 ♡J ♧2 ♤A ♢8 ♢K ♤4 ♢7
♧6 ♡K ♧J ♡6 ♢5 ♧K ♤10 ♤Q ♢10
♧5 ♤2 ♤3 ♢4 ♧9 ♧Q ♡A ♤8 ♤J
♤7 ♡10 ♢J ♡5 ♢Q ♡2 ♤6 ♡3 ♧10
♢3 ♢A ♡Q ♤K ♧7 ♤5 ♡4 ♢6 ♧4
Yes No Yes No Yes Yes
1


In [98]:
# Yesのみのケース
output_answer(57)

♡2 ♡K ♡3 ♧10 ♧4 ♧6 ♤J ♧K ♤4
♤8 ♢2 ♢7 ♢10 ♢J ♤7 ♤A ♢4 ♢3
♢6 ♢K ♧2 ♡4 ♢Q ♤5 ♧Q ♤3 ♧J
♧8 ☆ ♡8 ♧9 ♧5 ♡Q ♡6 ♡5 ♤Q
♤6 ♡A ♧A ♧3 ♤K ♤10 ♡9 ♢5 ♤9
♡7 ♧7 ♢A ♡J ♢9 ♡10 ♤2 ♢8
Yes Yes Yes Yes Yes Yes
4
